<h2>IMDB dateset</h2>

IMDB는 (영화 리뷰, 리뷰의 긍정부정 값)을 한 쌍의 데이터 값으로, 50,000개의 샘플을 모아 놓은 데이터 셋이다. 이 중 25,000개의 댓글 셈플은 트레이닝 샘플이며, 나머지 25,000개의 댓글 샘플은 테스팅 샘플이다.<br>
다시말해, 해당 데이터 셋에는 50000개의 영화 리뷰를 가지고 있으며, 모든 리뷰들은 결과 값(부정 혹은 긍적) 값을 결과 값으로 가지고 있다. 
해당 데이터 셋의 결과 값이 이진 데이터라는 특성 때문에 해당 데이터 셋을 사용하는 신경망 모델의 목적은 주로 binary classification을 목적으로 한다.

해당 파일에서는 편의를 위해 데이터를 로드할 때, 가장 빈번하게 사용되는 20,000개의 단어들만 포함하여 로드하였다. 

In [2]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

모델 내부에서 training을 위한 데이터와 이를 각 epoch에서 검증하기 위한 데이터를 분류
하기 위해 25,000개의 데이터를 분할하였다.<br>
25,000개이 샘플 중 20,000개는 트레이닝을 위한 샘플, 나머지 5,000개는 트레이닝 후 정확도 평가를 위한 validation 샘플로 두었다.

In [5]:
x_val=x_train[20000:]
y_val=y_train[20000:]

x_train=x_train[:20000]
y_train=y_train[:20000]

<h4>pad_sequence() function</h4>
pad_sequence()라는 함수는 케라스에 내장되어 있는 함수로, 데이터 전처리를 하는 것이 그 용도이다. </br>
해당 함수를 사용하면, 샘플 간의 길이를 균등하게 맞출 수 있다.

In [7]:
from keras.preprocessing import sequence

x_train=sequence.pad_sequences(x_train, maxlen=200) #길이 200으로 맞춤
x_val=sequence.pad_sequences(x_val, maxlen=200)
x_test=sequence.pad_sequences(x_test, maxlen=200)

<h2>모델 구성</h2>
Sequential() 함수로 모델을 초기화 한다.

<h4>Embedding()층을 추가한다</h4>
Embedding()은 one-hot-encoding보다 훨씬 적은 공간을 사용한다. 또한 해당 층을 추가하면 단어 간의 연관성을 고려한 벡터가 만들어지기 때문에 훨씬 유의미한 훈련이 가능하다.

<h4>Dropout</h4>
Dropout은 훈련 중에 과대적합을 방지하기 위해서 추가된 층이다.

<h4>Conv1D</h4>
Convolutional Neural Network(CNN)은 주로 2차원으로 사용할 때 이미지 훈련에서 빈번하게 사용된다. 하지만 이 외의 분류 모델에서도 사용이 용이하다.<br> 
CNN은 지정한 갯수만큼의 fileter(합성곱 윈도우)를 가지고 데이터가 가지고 있는 패턴을 찾아낸다. 이 때, 필터를 많이 가지고 있을수록, 더 다양한 패턴을 인식하게 된다. 그러나 연산량은 많아진다. 

<h4>GlobalMaxPooling</h4>
Pooling은 벡터에서 특징적인 값만 추출하는 과정이다. 특정한 벡터만 추출해서 사용하게 되면 과대적합 문제를 해결하는데에 도움을 줄 수 있다.<br>GlobalMaxPooling의 경우에는 여러 개의 벡터 정보 중에서 가장 큰 정보만 추출해오는 pooling 방법인다.  


<h4>출력층: Sigmoid 함수</h4>
해당 모델의 목적은 입력되는 리뷰의 긍정 부정을 판별하는 것이다. Binary classification에 적합한 Sigmoid 함수를 사용하였다.

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D

model=Sequential()
model.add(Embedding(20000, 128, input_length=200))
model.add(Dropout(0.2))
model.add(Conv1D(256, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 256)          98560     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 69s 3ms/step - loss: 0.4959 - acc: 0.7537 - val_loss: 0.3251 - val_acc: 0.8654
Epoch 2/25
20000/20000 [==============================] - 64s 3ms/step - loss: 0.2722 - acc: 0.8901 - val_loss: 0.2826 - val_acc: 0.8810
Epoch 3/25
20000/20000 [==============================] - 64s 3ms/step - loss: 0.1681 - acc: 0.9368 - val_loss: 0.2792 - val_acc: 0.8854
Epoch 4/25
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0894 - acc: 0.9714 - val_loss: 0.3079 - val_acc: 0.8846
Epoch 5/25
20000/20000 [==============================] - 63s 3ms/step - loss: 0.0419 - acc: 0.9884 - val_loss: 0.3703 - val_acc: 0.8802
Epoch 6/25
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0189 - acc: 0.9960 - val_loss: 0.4202 - val_acc: 0.8768
Epo

In [17]:
result=model.evaluate(x_test, y_test)
print("====Test Loss====")
print(result[0])
print("====Test Accuracy")
print(result[1])

25000/25000 [==============================] - 21s 833us/step
====Test Loss====
1.0296555969542265
====Test Accuracy
0.84088
